# New Age

Fakulteta za matematiko in fiziko - Univerza v Ljubljani

Ivan Slapničar, Sveučilište u Splitu, FESB

April, 2017

## Messages

Easy -> hard?

Present: __age of search__ -> _mathematics_

Future: __age of recommendation__  -> _mathematics_

(BigData, new technologies)


## Easy -> hard

weight:  1 _kg_  -> 1 _kg_ $\pm$ 0.000000001 _kg_

running: 100 _m_ -> 42,195 _m_ ili 100 _m_ < 10 _sek_

mathematics: exam -> state competition  -> [Olympiad](http://www.imo-official.org/)

search, recommending: good -> excellent

## Age of Search

google (and others)


* [50 billion pages](http://www.worldwidewebsize.com/), [3.5 billion querries daily](http://www.internetlivestats.com/google-search-statistics/)
* __PageRank__
* history, context - cookies, storing data (about you), [200+ parameters](http://backlinko.com/google-ranking-factors)

## PageRank

* Graph theory and linear algebra
* [C, Moler, Google PageRank][Mol11]


[Mol11]: https://www.mathworks.com/moler/exm/chapters/pagerank.pdf "C, Moler, 'Google PageRank', mathWorks, 2011."


In [1]:
i = vec([ 2 6 3 4 4 5 6 1 1])
j = vec([ 1 1 2 2 3 3 3 4 6])
G=sparse(i,j,1.0)

6×6 sparse matrix with 9 Float64 nonzero entries:
	[2, 1]  =  1.0
	[6, 1]  =  1.0
	[3, 2]  =  1.0
	[4, 2]  =  1.0
	[4, 3]  =  1.0
	[5, 3]  =  1.0
	[6, 3]  =  1.0
	[1, 4]  =  1.0
	[1, 6]  =  1.0

In [2]:
full(G)

6×6 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0

On the graph we do a _random walk_ - probabilty to follow any link is equal:

In [3]:
c=sum(G,1)
n=size(G,1)
for j=1:n
    if c[j]>0
        G[:,j]=G[:,j]/c[j]
    end
end
full(G)

6×6 Array{Float64,2}:
 0.0  0.0  0.0       1.0  0.0  1.0
 0.5  0.0  0.0       0.0  0.0  0.0
 0.0  0.5  0.0       0.0  0.0  0.0
 0.0  0.5  0.333333  0.0  0.0  0.0
 0.0  0.0  0.333333  0.0  0.0  0.0
 0.5  0.0  0.333333  0.0  0.0  0.0

* $p$: probability to follow link
* $1-p$: probability to visit some other page (at random)
* google uses $p=0.85$ ?

In [4]:
p=0.85
z = ((1-p)*(c.!=0) + (c.==0))/n
A=p*G+ones(n)*z

6×6 Array{Float64,2}:
 0.025  0.025  0.025     0.875  0.166667  0.875
 0.45   0.025  0.025     0.025  0.166667  0.025
 0.025  0.45   0.025     0.025  0.166667  0.025
 0.025  0.45   0.308333  0.025  0.166667  0.025
 0.025  0.025  0.308333  0.025  0.166667  0.025
 0.45   0.025  0.308333  0.025  0.166667  0.025

## Idea

Let us start a random walk from the vector $x_0=\begin{bmatrix} 1/n \\ 1/n \\ \vdots \\ 1/n \end{bmatrix}$.

The subsequent vectors are  

$$
x_1=A\cdot x_0 \\
x_2=A\cdot x_1 \\
x_3=A\cdot x_2\\
\vdots
$$

When the vector __stabilizes__:

$$
A\cdot x\approx x,
$$

then $x[i]$ is the __rank of the page__ $i$.

In [5]:
function myPageRank(G::SparseMatrixCSC{Float64,Int64},steps::Int)
    p=0.85
    c=sum(G,1)/p
    n=size(G,1)
    for i=1:n
        G.nzval[G.colptr[i]:G.colptr[i+1]-1]./=c[i]
    end
    e=ones(n)
    x=e/n
    z = vec(((1-p)*(c.!=0) + (c.==0))/n)
    for j=1:steps
        x=G*x+(z⋅x)
    end
    x/norm(x,1)
end

myPageRank (generic function with 1 method)

In [7]:
myPageRank(G,15)

6-element Array{Float64,1}:
 0.321024 
 0.170538 
 0.106596 
 0.136795 
 0.0643103
 0.200737 

In [6]:
fieldnames(G)

5-element Array{Symbol,1}:
 :m     
 :n     
 :colptr
 :rowval
 :nzval 

In [7]:
G

6×6 sparse matrix with 9 Float64 nonzero entries:
	[2, 1]  =  0.5
	[6, 1]  =  0.5
	[3, 2]  =  0.5
	[4, 2]  =  0.5
	[4, 3]  =  0.333333
	[5, 3]  =  0.333333
	[6, 3]  =  0.333333
	[1, 4]  =  1.0
	[1, 6]  =  1.0

In [8]:
G.colptr

7-element Array{Int64,1}:
  1
  3
  5
  8
  9
  9
 10

### [Stanford web graph](http://snap.stanford.edu/data/web-Stanford.html)

This is somewhat larger test problem.

In [9]:
W=readdlm("web-Stanford.txt",Int)

2312497×2 Array{Int64,2}:
      1    6548
      1   15409
   6548   57031
  15409   13102
      2   17794
      2   25202
      2   53625
      2   54582
      2   64930
      2   73764
      2   84477
      2   98628
      2  100193
      ⋮        
 281849  165189
 281849  177014
 281849  226290
 281849  243180
 281849  244195
 281849  247252
 281849  281568
 281865  186750
 281865  225872
 281888  114388
 281888  192969
 281888  233184

In [10]:
S=sparse(W[:,2],W[:,1],1.0)

281903×281903 sparse matrix with 2312497 Float64 nonzero entries:
	[6548  ,      1]  =  1.0
	[15409 ,      1]  =  1.0
	[17794 ,      2]  =  1.0
	[25202 ,      2]  =  1.0
	[53625 ,      2]  =  1.0
	[54582 ,      2]  =  1.0
	[64930 ,      2]  =  1.0
	[73764 ,      2]  =  1.0
	[84477 ,      2]  =  1.0
	[98628 ,      2]  =  1.0
	⋮
	[168703, 281902]  =  1.0
	[180771, 281902]  =  1.0
	[266504, 281902]  =  1.0
	[275189, 281902]  =  1.0
	[44103 , 281903]  =  1.0
	[56088 , 281903]  =  1.0
	[90591 , 281903]  =  1.0
	[94440 , 281903]  =  1.0
	[216688, 281903]  =  1.0
	[256539, 281903]  =  1.0
	[260899, 281903]  =  1.0

In [13]:
@time x100=myPageRank(S,100);

  1.663106 seconds (846.56 k allocations: 542.589 MB, 4.09% gc time)


In [14]:
x101=myPageRank(S,101)
maxabs((x100-x101)./x100)

2.3491380959929403e-7

In [15]:
sortperm(x100,rev=true), sort(x101,rev=true)

([89073,226411,241454,262860,134832,234704,136821,68889,105607,69358  …  281647,281700,281715,281728,281778,281785,281813,281849,281865,281888],[0.0113029,0.00926783,0.00829727,0.00302312,0.00300128,0.00257173,0.00245371,0.00243079,0.00239105,0.00236401  …  5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7,5.33369e-7])

## Age of Recommendation

NetFlix, Amazon Prime, PickBox, ... - on-line streaming of movies and shows

[NetFlix](https://www.netflix.com/hr/)

 * [80 million users](https://www.statista.com/statistics/250934/quarterly-number-of-netflix-streaming-subscribers-worldwide/), 5,000 movies
 * [NetFlix Prize](http://www.kdd.org/kdd2014/tutorials/KDD%20-%20The%20Recommender%20Problem%20Revisited.pdf)
 

## Mathematics

Netflix Recommendation Engine is based on approximation of a (large and sparse) matrix
```
M = Users x Movies 
```
using (approximation of) singular value decomposition (SVD): 

* [IncrementalSVD.jl](https://github.com/aaw/IncrementalSVD.jl)
* [A parallel recommendation engine in Julia](http://juliacomputing.com/blog/2016/04/22/a-parallel-recommendation-engine-in-julia.html)

## Similarities

Similarity of users $i$ and $k$:
$$
\cos \angle (i,k)=\frac{(M[i,:],M[k,:])}{\|M[i,:]\| \cdot \|M[k,:]\|}
$$
Similarity of movies $i$ and $k$:
$$
\cos \angle (i,k)=\frac{(M[:,i],M[:,k)}{\|M[:,i]\| \cdot \|M[:,k]\|}
$$

## Search

Row $M[u,:]$ - what user $u$ thinks about movies

Column $M[:,m]$ - what users think about movie $m$

Element $M[u,m]$ - what user $u$ thinks about movie $m$.

## Problem

Matrix $M$ is sparse so we do not have enough information. For example, 

```
900188 marks / ( 6040 users x 3706 movies ) = 4%
```

## Approximation

SVD decomposition $M=U\Sigma V^T$ is [approximated by a low-rank matrix](https://en.wikipedia.org/wiki/Low-rank_approximation) (for example. $rank=25$)

![SVD decomposition](svd.png)

The approximation matrix is __full__ and __gives enough good information__.

Prize for efficient approximation algorithm was $\$$ 1.000.000.  

In [16]:
# Pkg.clone("git://github.com/aaw/IncrementalSVD.jl.git")

In [17]:
using IncrementalSVD

INFO: Recompiling stale cache file C:\Users\Ivan\.julia\lib\v0.5\ProgressMeter.ji for module ProgressMeter.


In [18]:
whos(IncrementalSVD)

                IncrementalSVD     56 KB     Module
                        Rating    136 bytes  DataType
                     RatingSet    148 bytes  DataType
                  RatingsModel    160 bytes  DataType
             cosine_similarity      0 bytes  IncrementalSVD.#cosine_similarity
          get_predicted_rating      0 bytes  IncrementalSVD.#get_predicted_rati…
                 item_features      0 bytes  IncrementalSVD.#item_features
                   item_search      0 bytes  IncrementalSVD.#item_search
                         items      0 bytes  IncrementalSVD.#items
    load_book_crossing_dataset      0 bytes  IncrementalSVD.#load_book_crossing…
  load_large_movielens_dataset      0 bytes  IncrementalSVD.#load_large_moviele…
  load_small_movielens_dataset      0 bytes  IncrementalSVD.#load_small_moviele…
                          rmse      0 bytes  IncrementalSVD.#rmse
                 similar_items      0 bytes  IncrementalSVD.#similar_items
                 similar_us

In [19]:
rating_set = load_small_movielens_dataset();

Reusing existing downloaded files...


Loading ratings 100% Time: 0:00:04


In [20]:
fieldnames(rating_set)

4-element Array{Symbol,1}:
 :training_set 
 :test_set     
 :user_to_index
 :item_to_index

In [21]:
# The format is (user, movie, mark)
rating_set.training_set

900188-element Array{IncrementalSVD.Rating,1}:
 IncrementalSVD.Rating(284,187,3.0)  
 IncrementalSVD.Rating(293,259,5.0)  
 IncrementalSVD.Rating(4277,519,4.0) 
 IncrementalSVD.Rating(1939,727,5.0) 
 IncrementalSVD.Rating(778,19,5.0)   
 IncrementalSVD.Rating(1297,673,5.0) 
 IncrementalSVD.Rating(4460,1136,5.0)
 IncrementalSVD.Rating(1137,1406,3.0)
 IncrementalSVD.Rating(5077,1299,4.0)
 IncrementalSVD.Rating(4948,1107,4.0)
 IncrementalSVD.Rating(3430,862,4.0) 
 IncrementalSVD.Rating(3687,547,2.0) 
 IncrementalSVD.Rating(4305,344,4.0) 
 ⋮                                   
 IncrementalSVD.Rating(3011,768,4.0) 
 IncrementalSVD.Rating(5031,530,5.0) 
 IncrementalSVD.Rating(4285,1722,1.0)
 IncrementalSVD.Rating(825,150,5.0)  
 IncrementalSVD.Rating(1097,1691,2.0)
 IncrementalSVD.Rating(686,190,3.0)  
 IncrementalSVD.Rating(3481,1285,5.0)
 IncrementalSVD.Rating(3729,1417,4.0)
 IncrementalSVD.Rating(2700,259,5.0) 
 IncrementalSVD.Rating(5172,1857,2.0)
 IncrementalSVD.Rating(3551,150,4.0) 
 In

In [22]:
rating_set.test_set

100021-element Array{IncrementalSVD.Rating,1}:
 IncrementalSVD.Rating(533,1096,1.0) 
 IncrementalSVD.Rating(1882,353,2.0) 
 IncrementalSVD.Rating(1466,1675,3.0)
 IncrementalSVD.Rating(2304,739,4.0) 
 IncrementalSVD.Rating(4457,49,5.0)  
 IncrementalSVD.Rating(1752,1252,4.0)
 IncrementalSVD.Rating(5627,215,3.0) 
 IncrementalSVD.Rating(1698,316,5.0) 
 IncrementalSVD.Rating(4322,862,5.0) 
 IncrementalSVD.Rating(1224,259,5.0) 
 IncrementalSVD.Rating(3823,510,4.0) 
 IncrementalSVD.Rating(2529,749,3.0) 
 IncrementalSVD.Rating(1465,215,4.0) 
 ⋮                                   
 IncrementalSVD.Rating(3205,1191,3.0)
 IncrementalSVD.Rating(5019,2103,3.0)
 IncrementalSVD.Rating(4508,313,3.0) 
 IncrementalSVD.Rating(3940,2476,4.0)
 IncrementalSVD.Rating(2181,879,2.0) 
 IncrementalSVD.Rating(4547,353,3.0) 
 IncrementalSVD.Rating(4553,653,4.0) 
 IncrementalSVD.Rating(5841,332,3.0) 
 IncrementalSVD.Rating(2223,861,1.0) 
 IncrementalSVD.Rating(5787,410,3.0) 
 IncrementalSVD.Rating(3057,54,5.0)  
 In

In [23]:
# Users and their IDs
rating_set.user_to_index

Dict{AbstractString,Int32} with 6040 entries:
  "4304" => 4304
  "3935" => 3935
  "5422" => 5422
  "5734" => 5734
  "2243" => 2243
  "1881" => 1881
  "5425" => 5425
  "4209" => 4209
  "1907" => 1907
  "2923" => 2923
  "599"  => 599
  "2491" => 2491
  "5944" => 5944
  "228"  => 228
  "2590" => 2590
  "3697" => 3697
  "5031" => 5031
  "2579" => 2579
  "5551" => 5551
  "1880" => 1880
  "2562" => 2562
  "3215" => 3215
  "3991" => 3991
  "4652" => 4652
  "4088" => 4088
  ⋮      => ⋮

In [24]:
# Movies and their IDs
rating_set.item_to_index

Dict{AbstractString,Int32} with 3706 entries:
  "Fried Green Tomatoes (1991)"                                       => 594
  "Milk Money (1994)"                                                 => 1361
  "From Russia with Love (1963)"                                      => 729
  "House II: The Second Story (1987)"                                 => 1247
  "Held Up (2000)"                                                    => 3549
  "Missing in Action 2: The Beginning (1985)"                         => 2177
  "Murder, My Sweet (1944)"                                           => 996
  "Hidden, The (1987)"                                                => 981
  "Cable Guy, The (1996)"                                             => 669
  "Big Kahuna, The (2000)"                                            => 893
  "Addams Family Values (1993)"                                       => 1857
  "Farinelli: il castrato (1994)"                                     => 1945
  "Education of Little T

In [25]:
# We can extract the titles ...
keys(rating_set.item_to_index)

Base.KeyIterator for a Dict{AbstractString,Int32} with 3706 entries. Keys:
  "Fried Green Tomatoes (1991)"
  "Milk Money (1994)"
  "From Russia with Love (1963)"
  "House II: The Second Story (1987)"
  "Held Up (2000)"
  "Missing in Action 2: The Beginning (1985)"
  "Murder, My Sweet (1944)"
  "Hidden, The (1987)"
  "Cable Guy, The (1996)"
  "Big Kahuna, The (2000)"
  "Addams Family Values (1993)"
  "Farinelli: il castrato (1994)"
  "Education of Little Tree, The (1997)"
  "In God's Hands (1998)"
  "Last Man Standing (1996)"
  "Sixth Sense, The (1999)"
  "Star Maps (1997)"
  "Girl, Interrupted (1999)"
  "Stand by Me (1986)"
  "Rob Roy (1995)"
  "Caligula (1980)"
  "Flirting With Disaster (1996)"
  "Hook (1991)"
  "Institute Benjamenta, or This Dream People Call Human Life (1995)"
  ⋮

In [26]:
# or codes
values(rating_set.item_to_index)

Base.ValueIterator for a Dict{AbstractString,Int32} with 3706 entries. Values:
  594
  1361
  729
  1247
  3549
  2177
  996
  981
  669
  893
  1857
  1945
  2759
  2814
  1840
  39
  2670
  32
  85
  449
  2532
  1290
  657
  3397
  ⋮

In [27]:
# Which movies did the user "3000" grade?
user_ratings(rating_set, "3000")

93-element Array{Tuple{SubString{String},Float32},1}:
 ("Brazil (1985)",5.0)                         
 ("Babe (1995)",5.0)                           
 ("Gattaca (1997)",5.0)                        
 ("Defending Your Life (1991)",5.0)            
 ("Brothers McMullen, The (1995)",5.0)         
 ("Caddyshack (1980)",5.0)                     
 ("Princess Bride, The (1987)",5.0)            
 ("Rock, The (1996)",5.0)                      
 ("Time Bandits (1981)",5.0)                   
 ("When Harry Met Sally... (1989)",5.0)        
 ("Twelve Monkeys (1995)",5.0)                 
 ("Groundhog Day (1993)",5.0)                  
 ("One Flew Over the Cuckoo's Nest (1975)",5.0)
 ⋮                                             
 ("Twister (1996)",2.0)                        
 ("Death Becomes Her (1992)",2.0)              
 ("Total Recall (1990)",2.0)                   
 ("Young Sherlock Holmes (1985)",2.0)          
 ("Forrest Gump (1994)",2.0)                   
 ("Dirty Dancing (1987)",1.0)     

In [28]:
# Let us find the exact title and code for "Blade runner"
for k in keys(rating_set.item_to_index)
    if contains(k,"Blade")
        println(k)
    end
end

Sling Blade (1996)
Blade (1998)
Blade Runner (1982)
Some Folks Call It a Sling Blade (1993)


LoadError: UnicodeError: invalid character index

In [29]:
get(rating_set.item_to_index,"Blade Runner (1982)",0)

744

In [30]:
# Did the user "3000" grade "Blade Runner" ?
for k in user_ratings(rating_set,"3000")
    if contains(k[1],"Blade")
        println(k)
    end
end

("Blade Runner (1982)",4.0f0)


In [31]:
# Did the user "3000" grade "Citizen Kane" ?
for k in user_ratings(rating_set,"3000")
    if contains(k[1],"Citizen")
        println(k)
    end
end

In [32]:
# This takes about 2.5 minutes
model = train(rating_set, 25);

Computing truncated rank 25 SVD 100% Time: 0:02:34


In [33]:
fieldnames(model)

5-element Array{Symbol,1}:
 :user_to_index
 :item_to_index
 :U            
 :S            
 :V            

In [34]:
model.U

6040×25 Array{Float32,2}:
 0.0109136   0.0120564    0.0160181    0.00870599  …   0.0150979   0.013934  
 0.0143597   0.0108303    0.00952503   0.0100758       0.00687544  0.0109029 
 0.00974189  0.0085419    0.0166435    0.00741452      0.00828278  0.0131701 
 0.00680543  0.00669967   0.0151626    0.00418964      0.0127535   0.0109583 
 0.0136235   0.0158545    0.00185634   0.0150315       0.00109894  0.0104951 
 0.0111683   0.0161907    0.0151677    0.0159385   …   0.00845536  0.013711  
 0.00861757  0.00714385   0.0183328    0.00368803      0.0112091   0.0121351 
 0.0156814   0.0145832    0.00987226   0.0131809       0.00931192  0.00965002
 0.0132787   0.00887208   0.0112577    0.00782999      0.00938066  0.00877462
 0.0186663   0.0128824   -0.0105403    0.0250137       0.00610384  0.0111481 
 0.012899    0.0111345    0.00903686   0.0106912   …   0.0103233   0.0101588 
 0.00637085  0.00855026   0.0146385    0.00626682      0.0130673   0.016565  
 0.0127295   0.00995596   0.0122353   

In [35]:
model.S

25-element Array{Float32,1}:
 8190.77 
 2149.5  
  824.986
 1149.77 
  847.328
 1209.95 
  618.968
  386.108
  352.067
  262.127
  253.602
  338.198
  304.998
  220.552
  309.492
  336.532
  338.199
  188.984
  198.19 
  194.301
  169.297
  144.274
  134.29 
  118.472
   96.437

In [36]:
model.V

3706×25 Array{Float32,2}:
 0.0357048   0.00763865  0.0678461   …   0.000220664   0.00511578 
 0.0248908   0.0148034   0.0193536      -0.00587556    0.00740207 
 0.0303079   0.0132326   0.0306704       0.00168739    0.00508068 
 0.032306    0.0123712   0.0540161       0.0207233    -0.0115093  
 0.0313989   0.00772239  0.0559563      -0.000758386   0.00210363 
 0.0340505   0.00453273  0.0697202   …  -0.0109375     0.000534025
 0.030447    0.0081357   0.0296927       0.00599335    0.0028845  
 0.0332151   0.00420182  0.0507269       0.0132758     0.00742786 
 0.0289114   0.00788475  0.0233104       0.000205755   0.0244397  
 0.033651    0.00520378  0.0532696      -0.0230184     0.0320608  
 0.02988     0.00539655  0.0362236   …   0.00615703   -0.0199942  
 0.0169084   0.0272362   0.00967267      0.00696713    0.00617051 
 0.0258817   0.0195916   0.0125598       0.00481522    0.00531761 
 ⋮                                   ⋱                            
 0.00119731  0.00260581  0.00408055 

In [37]:
similar_items(model, "Friday the 13th (1980)")

10-element Array{SubString{String},1}:
 "Friday the 13th (1980)"                   
 "Amityville Horror, The (1979)"            
 "Jaws 2 (1978)"                            
 "Friday the 13th Part 2 (1981)"            
 "Halloween II (1981)"                      
 "Omen, The (1976)"                         
 "Pet Sematary (1989)"                      
 "Friday the 13th: The Final Chapter (1984)"
 "Cujo (1983)"                              
 "Creepshow (1982)"                         

In [38]:
@which similar_items(model, "Friday the 13th (1980)")

similar_items(model::IncrementalSVD.RatingsModel, item) at C:\Users\Ivan\.julia\v0.5\IncrementalSVD\src\util.jl:113

In [39]:
similar_items(model, "Citizen Kane (1941)")

10-element Array{SubString{String},1}:
 "Citizen Kane (1941)"                                                        
 "M*A*S*H (1970)"                                                             
 "Chinatown (1974)"                                                           
 "Rear Window (1954)"                                                         
 "Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)"
 "Lawrence of Arabia (1962)"                                                  
 "Manchurian Candidate, The (1962)"                                           
 "African Queen, The (1951)"                                                  
 "Cool Hand Luke (1967)"                                                      
 "Casablanca (1942)"                                                          

In [40]:
similar_users(model,"3000")

10-element Array{SubString{String},1}:
 "3000"
 "5613"
 "1360"
 "3845"
 "2202"
 "5619"
 "3052"
 "1409"
 "297" 
 "523" 

In [41]:
# What is the opinion of user "3000" about "Blader Runner" 
# in the approximate model (true mark is 4.0) ?
get_predicted_rating(model, "3000", "Blade Runner (1982)")

4.296962f0

In [42]:
# What is the opinion of user "3000" about Citizen Kane 
# (no true mark!) ?
IncrementalSVD.get_predicted_rating(model, "3000", "Citizen Kane (1941)")

4.267635f0

## Thank you for you attention

### Questions?